# Hybrid compute on Quantinuum devices via Nexus

#### N.B Not to be published until the QEC Decoder Toolkit announcement

Quantinuum offers real-time hybrid compute capability for your workflows. This capability executes Web Assembly (Wasm) in the H-Series stack (or Nexus-hosted H-series emulators) and enables use of libraries (e.g. linear algebra and graph algorithms) and complex datastructures (e.g. vectors and graphs) during real-time execution of a quantum circuit.

This notebook uses Nexus to submit a circuit with a very simple Wasm example.



In [1]:
import qnexus as qnx

from pytket.circuit import Circuit
from pytket.wasm import WasmFileHandler

from datetime import datetime


In [2]:
my_project_ref = qnx.projects.get_or_create(name="My Wasm Example Project")

qnx.context.set_active_project(my_project_ref)

Load our Wasm file and upload it to Nexus.

In [ ]:
wfh = WasmFileHandler(filepath="data/add_one.wasm")

print(repr(wfh))

wasm_module_ref = qnx.wasm_modules.upload(
    wasm_module_handler=wfh,
    name="Add One Wasm"
)

wasm_module_ref.df()

In [ ]:
circuit = Circuit(0, 2)
# Very minimal WASM example
circuit.add_wasm("add_one", wfh, [1], [1], [0, 1])

Upload our circuit to Nexus and compile it for our target Quantinuum device.

In [7]:
my_circuit_ref = qnx.circuits.upload(
    name=f"My example Add One WASM circuit",
    circuit = circuit,
)

In [8]:
backend_config = qnx.QuantinuumConfig(
    device_name="H1-Emulator",
)

In [ ]:
compiled_circuits = qnx.compile(
    circuits=[my_circuit_ref],
    name=f"My Compile Job from {datetime.now()}",
    optimisation_level=1,
    backend_config=backend_config,
)

compiled_circuits.df()

After compilation, we can submit the circuit to execute on the chosen device. The Wasm binary is stored seperately to the Circuit in Nexus, so we must submit the `WasmModuleRef` along with the circuit as part of the job.

In [ ]:
execute_job_ref = qnx.start_execute_job(
    circuits=[my_circuit_ref],
    name=f"My Execute Job from {datetime.now()}",
    n_shots=[100],
    backend_config=backend_config,
    wasm_module=wasm_module_ref,
)

execute_job_ref.df()

In [ ]:
qnx.jobs.wait_for(execute_job_ref)

Once the job result is complete we can download and analyse the data.

In [12]:
result_refs = qnx.jobs.results(execute_job_ref)
result = result_refs[0].download_result()

In [ ]:
result.get_counts()
